In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [14]:
df_max = df_receipt.groupby('customer_id').sales_ymd.max()
df_min = df_receipt.groupby('customer_id').sales_ymd.min()
df1 = pd.concat([df_max,df_min],axis=1)
df1.columns = ['sales_ymd_new','sales_ymd_old']
df1

,sales_ymd_new,sales_ymd_old
customer_id,,
CS001113000004,20190308,20190308
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
...,...,...
CS051513000004,20190719,20190719
CS051515000002,20191025,20191025
CS052212000002,20191017,20191017


In [21]:
df2 = df1[~(df1['sales_ymd_new'] == df1['sales_ymd_old'])]
df2.head(10)

,sales_ymd_new,sales_ymd_old
customer_id,,
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [45]:
df3 = df_receipt.groupby('store_cd').amount.mean().reset_index()
df3.head()

,store_cd,amount
0,S12007,307.688343
1,S12013,330.194130
2,S12014,310.830261
3,S12029,315.623908
4,S12030,288.533727


In [46]:
df3.sort_values(by='amount',ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [47]:
df4 = df_receipt.groupby('store_cd').amount.median().reset_index()
df4.sort_values(by='amount',ascending=False).head(5)

,store_cd,amount
28,S13052,190
30,S14010,188
51,S14050,185
44,S14040,180
7,S13003,180


P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [64]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index()

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。